In [ ]:
import logging
import sys
import os
from dotenv import load_dotenv
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

load_dotenv()

import llama_index
llama_index.__version__

In [1]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('assets').load_data()
index = VectorStoreIndex.from_documents(documents=documents, show_progress=True)

/opt/homebrew/Caskroom/miniforge/base/envs/llamaindex/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 109/109 [00:03<00:00, 29.71it/s]


In [30]:
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.postprocessor import SimilarityPostprocessor, KeywordNodePostprocessor
from llama_index.response_synthesizers import get_response_synthesizer


query_engine = index.as_query_engine()
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=4,
)
s_processor = SimilarityPostprocessor(similarity_cutoff=0.5) # 스코어 컷오프 지정
k_processor = KeywordNodePostprocessor(
    exclude_keywords=['cummecial'],
    # required_keywords=['llama2']
)
# 디버깅을 위해서 llm으로 보내지 않음(토큰 사용 x) 쿼리 엔진을 실행하면 final answer가 나오지 않음.
response_synthesize = get_response_synthesizer(
    response_mode='no_text' 
)
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[k_processor, s_processor],
    # response_synthesizer=response_synthesize
)

In [29]:
from llama_index.response.pprint_utils import pprint_response

response = query_engine.query('what is llama2?')
pprint_response(response, show_source=True) # 보여주기만 하는 기능
print(response)

Final Response: Llama 2 is a collection of pretrained and fine-tuned
large language models (LLMs) ranging in scale from 7 billion to 70
billion parameters. These models, called Llama 2-Chat, are optimized
for dialogue use cases and have demonstrated competitiveness with
existing open-source chat models. They are considered to be a suitable
substitute for closed-source models based on evaluations for
helpfulness and safety. Llama 2 is made available for both research
and commercial use, and developers must comply with the terms of the
provided license and the Acceptable Use Policy. The developers of
Llama 2 have also provided code examples and a Responsible Use Guide
to assist developers in safe development and deployment. The release
of Llama 2 is part of an open approach to encourage responsible AI
innovation and collaboration within the AI community.
______________________________________________________________________
Source Node 1/4
Node ID: ff89be49-96f5-4ad9-b2a4-0f07fed644cd
Si

In [31]:
# 그냥 응답의 모든 정보를 보려면
print(response.source_nodes)

[NodeWithScore(node=TextNode(id_='ff89be49-96f5-4ad9-b2a4-0f07fed644cd', embedding=None, metadata={'page_label': '36', 'file_name': 'llama2.pdf', 'file_path': 'assets/llama2.pdf', 'file_type': 'application/pdf', 'file_size': 13661300, 'creation_date': '2024-02-04', 'last_modified_date': '2023-12-16', 'last_accessed_date': '2024-02-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='6fa2b14b-261f-43bb-8201-1192e3dd2c8b', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '36', 'file_name': 'llama2.pdf', 'file_path': 'assets/llama2.pdf', 'file_type': 'application/pdf', 'file_size': 13661300, 'creation_date': '2024-02-04', 'last_modified_date': '2023-12-16', 'last_accessed_date': '2024-02-04'}, ha